In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# 首先使用 Pandas 库读入训练数据和测试数据，保存到 Train_data 和 Test_data 变量中。
Train_data = pd.read_csv(r'D:\AI_Learning\python\Neural_Networks\tianchi_competition\data\used_car_train_20200313.csv',
                         sep=' ')  # handle_used_car_train.csv
Test_data = pd.read_csv(r'D:\AI_Learning\python\Neural_Networks\tianchi_competition\data\used_car_testB_20200421.csv', sep=' ')

# 使用 pd.concat() 函数将训练数据和测试数据合并，并保存到 df 变量中。
df = pd.concat([Train_data, Test_data], ignore_index=True)

In [3]:
# 对 'price' 做对数变换，使用 np.log1p() 函数。
df['price'] = np.log1p(df['price'])

In [4]:
# 用众数填充缺失值
df['fuelType'] = df['fuelType'].fillna(0)
df['gearbox'] = df['gearbox'].fillna(0)
df['bodyType'] = df['bodyType'].fillna(0)
df['model'] = df['model'].fillna(0)

# 处理异常值，主要是将功率大于 600 的标为 600
df['power'] = df['power'].map(lambda x: 600 if x > 600 else x)  # 赛题限定power<=600
# 将 ‘notRepairedDamage’ 中的缺失值替换为 None
df['notRepairedDamage'] = df['notRepairedDamage'].astype('str').apply(lambda x: x if x != '-' else None).astype(
    'float32')

# 对可分类的连续特征进行分桶，例如将功率（power）分为 31 组，车型（model）分为 24 组。
bin = [i * 10 for i in range(31)]
df['power_bin'] = pd.cut(df['power'], bin, labels=False)

bin = [i * 10 for i in range(24)]
df['model_bin'] = pd.cut(df['model'], bin, labels=False)

In [ ]:
# 对日期数据进行处理，主要是提取年，月，日等信息和计算二手车使用时间。
from datetime import datetime

# 把原始的日期字符串或整数格式（如 20160405）转换为标准的 datetime 类型（如 2016-04-05）。
def date_process(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])

    if month < 1:
        month = 1

    date = datetime(year, month, day)
    return date

# 从注册日期/创建日期中提取：
# 年（如 2016）
# 月（如 4）
# 日（如 5）
# 分别存入新列 regDate_year、regDate_month、regDate_day
df['regDate'] = df['regDate'].apply(date_process)
df['creatDate'] = df['creatDate'].apply(date_process)
df['regDate_year'] = df['regDate'].dt.year
df['regDate_month'] = df['regDate'].dt.month
df['regDate_day'] = df['regDate'].dt.day
df['creatDate_year'] = df['creatDate'].dt.year
df['creatDate_month'] = df['creatDate'].dt.month
df['creatDate_day'] = df['creatDate'].dt.day

# 使用天数
df['car_age_day'] = (df['creatDate'] - df['regDate']).dt.days
# 使用年数
df['car_age_year'] = round(df['car_age_day'] / 365, 1)

# 对行驶路程和功率数据进行统计，例如：计算行驶路程与功率的最大值、最小值、中位数和均值等。
# 对训练数据中 kilometer（行驶里程） 分组后，统计每组对应的 power（发动机功率）的一些描述性统计特征，并将这些统计特征合并回主数据 df 中。
# kk[0] = 'kilometer'：表示按这个字段分组（行驶里程）
# kk[1] = 'power'：表示对这个字段计算统计量（功率）
kk = ['kilometer', 'power']
# 对 Train_data 按照 kilometer 分组；对每组的 power 值计算多个统计量：
t1 = Train_data.groupby(kk[0], as_index=False)[kk[1]].agg(
    {kk[0] + '_' + kk[1] + '_count': 'count', # 每个里程值对应多少条记录
     kk[0] + '_' + kk[1] + '_max': 'max', # 最大功率
     kk[0] + '_' + kk[1] + '_median': 'median', # 中位功率
     kk[0] + '_' + kk[1] + '_min': 'min',  # 最小功率
     kk[0] + '_' + kk[1] + '_sum': 'sum',  # 总功率
     kk[0] + '_' + kk[1] + '_std': 'std', # 功率标准差
     kk[0] + '_' + kk[1] + '_mean': 'mean'}) # 	平均功率
df = pd.merge(df, t1, on=kk[0], how='left') # 把这些统计特征 t1 按照 kilometer（行驶里程）合并回主表 df。合并方式是 left join，保留 df 中所有行。

C:\Users\Administrator\AppData\Local\Temp\ipykernel_76316\1521240310.py:33: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  t1 = Train_data.groupby(kk[0], as_index=False)[kk[1]].agg(


In [ ]:
# 为部分属性列的数据生成新的特征，主要是通过对 V0、V3、V8 和 V12 四个特征进行组合生成新的二元和三元特征。
num_cols = [0, 3, 8, 12]
for i in num_cols:
    for j in num_cols:
        df['new' + str(i) + '*' + str(j)] = df['v_' + str(i)] * df['v_' + str(j)]

for i in num_cols:
    for j in num_cols:
        df['new' + str(i) + '+' + str(j)] = df['v_' + str(i)] + df['v_' + str(j)]

for i in num_cols:
    for j in num_cols:
        df['new' + str(i) + '-' + str(j)] = df['v_' + str(i)] - df['v_' + str(j)]

for i in range(15):
    df['new' + str(i) + '*year'] = df['v_' + str(i)] * df['car_age_year']
# 这段代码的目标是扩展原始特征空间，帮助模型学习更复杂的特征关系。
# 类型	    示例	            意图
# 乘法组合	v_0 * v_3	        捕捉非线性关系
# 加法组合	v_8 + v_12	        提供总量特征
# 减法组合	v_12 - v_0	        提供差异特征
# 年龄交互	v_5 * car_age_year	考虑特征随时间变化的影响    


In [ ]:
# 导入tqdm库
from tqdm import tqdm

# 使用LightGBM 回归器作为模型，对数据进行训练和预测。
# 对数据进行五折交叉检验，最后通过将五次模型训练得到的结果平均作为最终预测结果，并将结果保存到文件中供提交。
df1 = df.copy()
test = df1[df1['price'].isnull()]
X_train = df1[df1['price'].notnull()].drop(['price', 'regDate', 'creatDate', 'SaleID', 'regionCode'], axis=1)
Y_train = df1[df1['price'].notnull()]['price']
X_test = df1[df1['price'].isnull()].drop(['price', 'regDate', 'creatDate', 'SaleID', 'regionCode'], axis=1)
# 五折交叉检验
cols = list(X_train) # 获取所有特征名
oof = np.zeros(X_train.shape[0]) # 保存 Out-Of-Fold 验证结果（每一折的验证集预测值）
sub = test[['SaleID']].copy() # 初始化提交结果 DataFrame
sub['price'] = 0  # 初始化预测价格为0
feat_df = pd.DataFrame({'feat': cols, 'imp': 0}) # 初始化特征重要性表
# 每次迭代用 3 折训练，1 折验证。
skf = KFold(n_splits=4, shuffle=True) #, random_state=2025

# 导入早停回调函数，如果在验证集上连续若干轮（比如 50 轮）没有提升，就提前停止训练，避免过拟合。
from lightgbm.callback import early_stopping

clf = LGBMRegressor(
    n_estimators=100000,        # 允许最多训练这么多棵树（配合早停避免过拟合）
    learning_rate=0.07,         # 0.02,    学习率，较小可使模型更稳定
    boosting_type='gbdt',       # 使用梯度提升树
    objective='regression_l1',  # 回归目标函数，L1损失（即 MAE）
    max_depth=-1,               # 不限制树的最大深度
    num_leaves=31,              # 每棵树最多的叶子节点数（越大模型越复杂）
    min_child_samples=20,       #  一个叶子上最少的数据数，用于防止过拟合
    feature_fraction=0.8,       # 每次训练随机使用 80% 的特征
    bagging_freq=1,             # 每轮都进行 bagging
    bagging_fraction=0.8,       #  每轮训练使用 80% 的样本
    lambda_l2=2,                # 	L2 正则项，防止过拟合
    # random_state=2025,
    metric='mae'                # 验证指标为 MAE（平均绝对误差）
)

mae = 0
# 使用tqdm包装循环，添加进度条
for i, (trn_idx, val_idx) in enumerate(tqdm(list(skf.split(X_train, Y_train)), desc="交叉验证进度")):
    print('--------------------- 第 {} 折 ---------------------'.format(i + 1))
    # trn_x / trn_y：训练数据和标签
    # val_x / val_y：验证数据和标签
    # .reset_index(drop=True) 是为了避免索引错乱
    trn_x, trn_y = X_train.iloc[trn_idx].reset_index(drop=True), Y_train[trn_idx]
    val_x, val_y = X_train.iloc[val_idx].reset_index(drop=True), Y_train[val_idx]
    # 用训练集训练模型
    # eval_set 指定验证集用于监控指标（MAE）
    # early_stopping(300)：如果 300 次迭代验证集的 MAE 没有提升，就提前停止训练
    clf.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)],
        eval_metric='mae',
        callbacks=[early_stopping(300)],  # 使用回调函数替代early_stopping_rounds
    )

    sub['price'] += np.expm1(clf.predict(X_test)) / skf.n_splits 
    # 对测试集进行预测
    # np.expm1(...) 是对预测值 还原对数变换，因为原标签可能之前做过 log1p（对价格取对数）
    # / skf.n_splits 是取平均预测（因为是每一折都预测了一次测试集，最后要平均）
    oof[val_idx] = clf.predict(val_x)
    mae += mean_absolute_error(np.expm1(val_y), np.expm1(oof[val_idx])) / skf.n_splits
    # 计算这一折的 MAE（使用还原后的价格）
    # mae 累加每一折的误差（最后是 4 折的平均）
print('交叉验证 MAE:', mae)

交叉验证进度:   0%|          | 0/4 [00:00<?, ?it/s]

--------------------- 第 1 折 ---------------------
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Auto-choosing col-wise multi-threa

交叉验证进度:  25%|██▌       | 1/4 [12:28<37:26, 748.72s/it]

--------------------- 第 2 折 ---------------------
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Auto-choosing col-wise multi-threa

交叉验证进度:  50%|█████     | 2/4 [22:04<21:34, 647.21s/it]

--------------------- 第 3 折 ---------------------
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Auto-choosing col-wise multi-threa

交叉验证进度:  75%|███████▌  | 3/4 [31:40<10:14, 614.49s/it]

--------------------- 第 4 折 ---------------------
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Auto-choosing col-wise multi-threa

交叉验证进度: 100%|██████████| 4/4 [39:15<00:00, 588.91s/it]

交叉验证 MAE: 460.91733166528695


In [ ]:
# 生成提交文件
sub.to_csv('submit_norandom.csv', index=False) # 不保存行索引列，只保存 DataFrame 的列（即只导出 SaleID 和 price 两列）